In [5]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine-date')

#와인 데이터 불러들이기

In [6]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()#데이터
target = wine['class'].to_numpy()#타겟

검증세트

In [7]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

#훈련데이터,테스트데이터 분류하기

In [8]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

#훈련데이터를 sub와 val로 나눔 sub->모델훈련, val->매게변수 튜닝
#마지막에 합친후 최종모델 훈련 -> 테스트 데이터 입력

In [9]:
print(sub_input.shape, val_input.shape) #test_size = 0.2니깐

(4157, 3) (1040, 3)


결정트리

In [11]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

#과대적합, 매개변수를 바꿀 필요가 있다

0.9971133028626413
0.864423076923077


교차검증

In [12]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

#기본적으로는 5개의 모델을 만듬

{'fit_time': array([0.00830817, 0.00698686, 0.00738001, 0.00760245, 0.00717926]), 'score_time': array([0.00081468, 0.00065088, 0.00067568, 0.00066805, 0.00070477]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [14]:
import numpy as np

print(np.mean(scores['test_score']))
#scor-testdml 5개 값을 평균냄

0.855300214703487


검증세트, 훈련세트 : 딥러닝

교차검증, 그리드서치 : 머신러닝

분할기(spliter)를 사용한 교차검증

In [15]:
from sklearn.model_selection import StratifiedKFold
#cv=10으로 폴더수 구분가능..10개 쓰고 싶다면
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

#cross_validate애서 cv객체는 분류모델인경우 StratifiedKFold()
#회귀모델일경우 kFlold()

0.855300214703487


In [16]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

#dt와 같은 객체를 통해 모델인식
#spliiter라는 객체를 통해 폴더를 나누는 행위를 제어 가능(훈련세트를 섞고 10개 폴드로 나눔)

0.8574181117533719


하이퍼파라미터튜닝

그리드서치: 교차검증+여러개의 매개변수

In [17]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}#넣을값 리스트 전체적으로는 딕셔너리
#분류,회귀시 많이 사용
#매개변수를 딕셔너리로 지정 ->최적값찾기
#기본5개모델 X 매개변수(5) = 25개모델
#불순도 차이= (부모노드 - 자식노드)불순도 = 정보이득 = 클수록 좋음 = 최솟값지정 => 가지치기유무

In [18]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
#n_jobs=-1 가능한 모든 코어를 다 사용

In [19]:
gs.fit(train_input, train_target)#25개 모델 훈련

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
    

In [20]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

#훈련종료시 가장 높은 모델로 다시 훈련

0.9615162593804117


In [21]:
print(gs.best_params_)
#최적의 매개변수 확인

{'min_impurity_decrease': 0.0001}


In [22]:
print(gs.cv_results_['mean_test_score'])
#교차검증을 점수는 gs.cv_results의 mean_test_score속성에 들어감

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [23]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])#mean_test_score의 최적값 인덱스
print(gs.cv_results_['params'][best_index])#그 인덱스를 활용해 최적매개변수 찾기

{'min_impurity_decrease': 0.0001}


매개변수를 더 많이 탐색해 보고 싶다 ..??

In [24]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),#9개 #매개변수불순도
          'max_depth': range(5, 20, 1),#15개 #매개변수 깊이
          'min_samples_split': range(2, 100, 10)#10개 노드에 분할하기 위한 최소한 샘플개수
          }
#9X15X10X5=6750
#결정트리와 관련되어있다<가지치기의 일환>

In [25]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'm

In [26]:
print(gs.best_params_)

#최적의 매개변수 조합

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [27]:
print(np.max(gs.cv_results_['mean_test_score']))

#최적의 교차검증점수

0.8683865773302731


확률분포선택<랜덤서치>

In [28]:
from scipy.stats import uniform, randint

#균등분포 -> 랜덤값 선택

In [29]:
rgen = randint(0, 10)
rgen.rvs(10)

array([1, 5, 5, 1, 5, 7, 7, 4, 0, 4])

In [30]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 99, 105,  91, 107, 101,  95, 107, 100,  91, 104]))

In [31]:
ugen = uniform(0, 1)
ugen.rvs(10)

#uniform 실수값 가능

array([0.90918114, 0.76095007, 0.45506757, 0.54490802, 0.10798245,
       0.07416922, 0.32938556, 0.3622878 , 0.03926237, 0.56525043])

In [32]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

#그리드서치 ->일정간격으로 테스트, 촘촘->모델이 많아짐
#범위를 크게주고 랜덤으로 테스트, 모델개수 규정가능-> 편리
#매개변수가 실수인경우도 효율적!

In [33]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

#샘플링은 100번

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=42,
         

In [34]:
print(gs.best_params_)

#최적의 매개변수

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [35]:
print(np.max(gs.cv_results_['mean_test_score']))

#검증 점수 

0.8695428296438884


In [36]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

#테스트세트 정확도 확인

0.86


요약

훈련세트 - 검증세트 - 테스트세트

검증세트를 이용해 모델 튜닝 교차검증을 사용해 훈련세트를 많이 사용

그리드서치 ->교차검증,매개변수 탐색을 한번에 실시

매개변수가 많은경우 너무 많은 모델이 만들어져
랜덤서치를 사용하여 모델의 수를 지정가능 + 넓은 범위의 매개변수 수행